IMPORTS AND CONFIGURATION

In [54]:
import numpy as np
import pandas as pd
import os
import importlib
import sys
import copy
import diskcache

In [55]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("The backtesting folder was not in the systems path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

path before ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\vinayak\\Desktop\\Backtesting']
path after ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pyt

In [56]:
# from scipy.optimize import minimize
from Modules import Plot
from Modules import Helpers
from Modules import Data as data
from Modules import Utility as util
from Modules import TradeAndLogics as TL
from Modules import Data_Processing as dp
from Modules.enums import Option, LongShort, DB, FNO, Leg, OHLC, PTSL

In [57]:
pwd()

'c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\Optimizations\\Intraday_ShortIC'

In [58]:
original_stdout = util.const(sys.stdout)
original_stdout.value

TRADING/ STRATEGY PARAMETERS

In [59]:
index_symbol = 'BANKNIFTY'
start_date = '2024-10-04'
end_date = '2024-11-17'
expiry_type_near = 'I'
expiry_type_next = 'II'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 1 # mins
look_back_window = 25*4
# Trade/ Strategy Parameters
buying_delta_threshold_per_lot = 5
selling_delta_threshold_per_lot = 1
notional_vega_buying = 10000 # rs
notional_vega_selling = 8000 # rs
profit_target = 100000
stop_loss = 100000
zscore_threshold_long = 2
zscore_threshold_short = -2
ic_threshold_long = 0.8
ic_threshold_short = 0.2
epsilon = 0.1
ICView = LongShort.Short
moneyness_ironfly = 0
price_factor = 5

In [60]:
def tell_delta_threshold_per_lot(ticker):
    if ticker.intention == LongShort.Long:
        return buying_delta_threshold_per_lot
    return selling_delta_threshold_per_lot

def tell_notional_vega(ticker):
    if ticker.intention == LongShort.Long:
        return notional_vega_buying
    return notional_vega_selling

In [61]:
strategy_desc = f"{ICView.name}IC_Intraday_Volatility_Arbitrage_{index_symbol}"

Strategy Specific Data Dealing and Modifications

In [62]:
def insert_index_component(constituents, constituent_symbol, weight, lot_size):
    dict = {
        'weight': weight,
        'lot_size': lot_size
    }
    constituents[constituent_symbol] = dict

In [63]:
constituents_weight_lots = {}

In [64]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

In [65]:
# insert_index_component(constituents_weight_lots, "PNB", 0.91, 8000)
insert_index_component(constituents_weight_lots, "ICICIBANK", 23.03, 700)
# insert_index_component(constituents_weight_lots, "AUBANK", 2.69, 1000)
# insert_index_component(constituents_weight_lots, "BANKBARODA", 1.84, 2925)
# insert_index_component(constituents_weight_lots, "FEDERALBNK", 1.68, 5000)
# insert_index_component(constituents_weight_lots, "IDFCFIRSTB", 1.08, 7500)
insert_index_component(constituents_weight_lots, "SBIN", 11.27, 750)
# insert_index_component(constituents_weight_lots, "INDUSINDBK", 5.58, 500)
insert_index_component(constituents_weight_lots, "HDFCBANK", 27.04, 550)
insert_index_component(constituents_weight_lots, "KOTAKBANK", 11.72, 400)
# insert_index_component(constituents_weight_lots, "BANDHANBNK", 1.98, 2800)
insert_index_component(constituents_weight_lots, "AXISBANK", 11.18, 625)

In [66]:
constituents_weight_lots

{'ICICIBANK': {'weight': 23.03, 'lot_size': 700},
 'SBIN': {'weight': 11.27, 'lot_size': 750},
 'HDFCBANK': {'weight': 27.04, 'lot_size': 550},
 'KOTAKBANK': {'weight': 11.72, 'lot_size': 400},
 'AXISBANK': {'weight': 11.18, 'lot_size': 625}}

In [67]:
def normalize_constituents_weights(constituents_weight_lots):
    weight_sum = 0
    for values in constituents_weight_lots.values():
        weight_sum += values['weight']
    if weight_sum != 100:
        print(f"Sum of weights = {weight_sum} is not 100. Normalizing weights...")
        total_weight = sum([values['weight'] for values in constituents_weight_lots.values()])
        for values in constituents_weight_lots.values():
            values['weight'] = (values['weight'] / total_weight) * 100
    
        normalized_sum = sum([values['weight'] for values in constituents_weight_lots.values()])
        print("Normalized sum:", normalized_sum)
        print(constituents_weight_lots)
    else:
        print(f"provided weights add up to perfect {weight_sum}")
    return (constituents_weight_lots)

In [68]:
normalize_constituents_weights(constituents_weight_lots)

Sum of weights = 84.24000000000001 is not 100. Normalizing weights...
Normalized sum: 100.00000000000001
{'ICICIBANK': {'weight': 27.338556505223178, 'lot_size': 700}, 'SBIN': {'weight': 13.378442545109213, 'lot_size': 750}, 'HDFCBANK': {'weight': 32.098765432098766, 'lot_size': 550}, 'KOTAKBANK': {'weight': 13.912630579297247, 'lot_size': 400}, 'AXISBANK': {'weight': 13.271604938271606, 'lot_size': 625}}


{'ICICIBANK': {'weight': 27.338556505223178, 'lot_size': 700},
 'SBIN': {'weight': 13.378442545109213, 'lot_size': 750},
 'HDFCBANK': {'weight': 32.098765432098766, 'lot_size': 550},
 'KOTAKBANK': {'weight': 13.912630579297247, 'lot_size': 400},
 'AXISBANK': {'weight': 13.271604938271606, 'lot_size': 625}}

NEAR MONTH (I) DATA FETCHING AND PROCESSING

In [69]:
import Dispersion.DispersionAdjustedFunctionality as daf

In [70]:
constituents_near = {}

In [71]:
logs_near = open(f'LOGS_Fetching_Near_Month_Data_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_near

index_near = daf.ticker(index_symbol, index_lot_size, True, start_date, end_date, expiry_type_near, True, timeframe, True, 0.1)
index_near.initializeDispersion(constituents_near, False, 1, ICView)
for stock, weight_lots in constituents_weight_lots.items():
    constituents_near[stock] = daf.ticker(stock, weight_lots['lot_size'], True, start_date, end_date, expiry_type_near, True, timeframe, False, 0.1)
    constituents_near[stock].initializeDispersion({}, True, weight_lots['weight']/100, ICView.opposite())

sys.stdout = original_stdout.value
logs_near.close()

In [72]:
ohlc = OHLC.close

In [73]:
index_near.set_ohlc(ohlc)
for _, constituent in index_near.components.items():
    constituent.set_ohlc(ohlc)

OHLC for BANKNIFTY is set to OHLC.close
OHLC for ICICIBANK is set to OHLC.close
OHLC for SBIN is set to OHLC.close
OHLC for HDFCBANK is set to OHLC.close
OHLC for KOTAKBANK is set to OHLC.close
OHLC for AXISBANK is set to OHLC.close


NEXT MONTH (II) DATA FETCHING AND PROCESSING

In [74]:
# constituents_next = {}

In [75]:
# logs_next = open(f'LOGS_Fetching_Next_Month_Data_for_{strategy_desc}.txt', 'w')
# sys.stdout = logs_next

# index_next = daf.ticker(index_symbol, index_lot_size, True, start_date, end_date, expiry_type_next, True, timeframe, True, 0.1)
# index_next.initializeDispersion(constituents_next, False, 1, ICView)
# for stock, weight_lots in constituents_weight_lots.items():
#     constituents_next[stock] = daf.ticker(stock, weight_lots['lot_size']/100, True, start_date, end_date, expiry_type_next, False, timeframe, False, 0.1)
#     constituents_next[stock].initializeDispersion({}, True, weight_lots['weight'], ICView.opposite())

# sys.stdout = original_stdout.value
# logs_next.close()

In [76]:
# ohlc = OHLC.close

In [77]:
# index_next.set_ohlc(ohlc)
# for _, constituent in index_next.components.items():
#     constituent.set_ohlc(ohlc)

NORMALIZE HORIZONTALLY OVER HERE

In [78]:
# def get_index_with_normalized_iv(index_near, index_next):
#     logs_normalization_iv = open(f'LOGS_IV_Normalization_for_{strategy_desc}.txt', 'w')
#     sys.stdout = logs_normalization_iv
#     index_fresh = copy.deepcopy(index_near)
#     print("Normallizing IV horizontally using Near and Next month options")
#     for timestamp in index_near.df_futures.index:
#         print(f"Timestamp: {timestamp}")
#         index_fresh.df_futures.loc[timestamp, 'iv'] = get_normallized_iv(timestamp, index_near, index_next)
#         for component_near, component_next in zip(index_near.components.values(), index_next.components.values()):
#             index_fresh.components[component_near.symbol].df_futures.loc[timestamp, 'iv'] = get_normallized_iv(timestamp, component_near, component_next)
#         print('-------------------------------------------------------------')
#     sys.stdout = original_stdout.value
#     logs_normalization_iv.close()
#     return index_fresh

In [79]:
index = copy.deepcopy(index_near)

BACK TO NORMAL

In [80]:
# logs_ic = open(f'LOGS_IC_Generation_for{strategy_desc}.txt', 'w')
# sys.stdout = logs_ic
# index.generate_ic_data()
# sys.stdout = original_stdout.value
# logs_ic.close()

In [81]:
# index.df_futures['filled_ic'] = index.df_futures['ic'].ffill()
# # index.df_futures['ic'] = index.df_futures['ic'].replace({pd.NA: np.nan})
# window = 500
# rolling_mean = index.df_futures['filled_ic'].rolling(window=window, min_periods=window).mean()
# rolling_std = index.df_futures['filled_ic'].rolling(window=window, min_periods=window).std()
# index.df_futures['zscore'] = (index.df_futures['filled_ic'] - rolling_mean) / rolling_std


In [82]:
# zscore = index.df_futures['zscore']
# ddf = pd.DataFrame({'zscore_smoothened': zscore.ewm(span=5).mean(), 'zscore': zscore, 'ema long': zscore.ewm(span=100).mean(), 'ema short': zscore.ewm(span=40).mean()})
# smooth_zscore = Plot.plot_df(ddf, 'zscore_smoothened', 'zscore', 'ema long', 'ema short')
# smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, -2, 0, len(ddf), 'Red', 1)
# smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, zscore_threshold_short, 0, len(ddf), 'Blue', 1)
# smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, zscore_threshold_long, 0, len(ddf), 'Blue', 1)
# smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, 2, 0, len(ddf), 'Red', 1)
# smooth_zscore.show()
# Plot.save_plot(smooth_zscore, f'PLOT_smoothened_zscore_for_{strategy_desc}.html')

In [83]:
# iv_df = []
# iv_df.append(index.df_futures['iv'].rename(index.symbol))
# for component in index.components.values():
#     iv_df.append(component.df_futures['iv'].rename(component.symbol))
# iv_df = pd.concat(iv_df, axis=1)
# fig_iv = Plot.plot_df(iv_df*100, *iv_df.columns)
# Plot.save_plot(fig_iv, f'PLOT_Normalized_IV_for_{strategy_desc}.html')
# fig_iv.show()

In [84]:
# def generate_signal_data(timestamp, index, index_near, index_next):
#     index_near.get_iv_at(timestamp)
#     index_next.get_iv_at(timestamp)
#     index.df_futures.loc[timestamp, 'iv'] = Helpers.get_normallized_iv(timestamp, index_near, index_next)
#     index.get_ic_at(timestamp)
#     window = 3000
#     idx = index.timestamps.get_loc(timestamp)
#     window_data = index.df_futures['ic'].iloc[idx - window + 1 : idx + 1]
#     rolling_mean = window_data.mean()
#     rolling_std = window_data.std()
#     zscore = (index.df_futures.loc[timestamp, 'ic'] - rolling_mean) / rolling_std
#     index.df_futures.loc[timestamp, 'zscore'] = zscore
#     return

In [85]:
def get_vega_legs(ticker, timestamp, *Legs):
    vega = 0
    for leg in Legs:
        if leg.Instrument == FNO.FUTURES:
            continue
        greeks = ticker.greeks(timestamp, leg.Instrument, leg.Strike)
        if greeks is None:
            return None
        vega += greeks['vega'] * leg.Position.value
    return abs(vega)

In [86]:
# def get_lots_for_neutrality(logging_information, vegas, weights, notional_vega):    
#     lots_for_neutrality = []
#     for symbol, _ in vegas.items():
#         lots = notional_vega * weights[symbol] / vegas[symbol]
#         lots_for_neutrality.append(lots)
#         logging_information[symbol]['Lots to take position'] = int(lots)
#     return int(np.round(lots_for_neutrality[0])), np.round(lots_for_neutrality[1:]).astype(int)

In [87]:
def get_proportionality_lots(logging_information, ticker, ticker_value, ticker_weight, notional_scaling_value):
    lots_ticker = abs(np.round(notional_scaling_value * ticker_weight/ ticker_value))
    logging_information[ticker.symbol]['Lots to take position'] = lots_ticker
    logging_information[ticker.symbol]['Vega Taken'] = lots_ticker * ticker_value
    return lots_ticker

In [88]:
def entry_signal(timestamp):
    if timestamp.hour == 9 and timestamp.minute == 20:
        return True
    return False
def exit_signal(timestamp):
    if timestamp.hour == 15 and timestamp.minute == 20:
        return True
    return False

In [89]:
def get_lots_for_entry(ticker, timestamp, **kwargs):
    logging_information = {}
    legs = Helpers.get_legs_ironfly_price(ticker, timestamp, 1, ticker.intention, moneyness_ironfly, price_factor)
    vega_ticker_ironfly = get_vega_legs(ticker, timestamp, *legs)
    if vega_ticker_ironfly is None:
        return None, None
    unweighted_vega = vega_ticker_ironfly * ticker.lot_size
    notional_vega = tell_notional_vega(ticker)
    idea_for_weight = ticker.weight
    logging_information[ticker.symbol] = {
        'Lot Size': ticker.lot_size,
        'Weight': ticker.weight,
        'Vega per IronFly': vega_ticker_ironfly,
        'Vega per Lot IronFly': vega_ticker_ironfly * ticker.lot_size,
        'Target Notional Vega': notional_vega * ticker.weight
    }
    
    # if 'vega_neutral' in kwargs and kwargs['vega_neutral']:
    
    if 'ic_neutral' in kwargs and kwargs['ic_neutral'] == True:
        if ticker.is_component:
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    elif 'theta_neutral' in kwargs and kwargs['theta_neutral'] == True:
        strike_ticker, _ = ticker.find_moneyness_strike(timestamp, 0, Option.Call)
        logging_information[ticker.symbol]['ATM Strike'] = strike_ticker
        if ticker.is_component:
            import math
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    lots_ticker = get_proportionality_lots(logging_information, ticker, unweighted_vega, idea_for_weight, notional_vega)
    print(f"{ticker.symbol} || Lots: {logging_information[ticker.symbol]['Lots to take position']}")
    for info_key, info_value in logging_information[ticker.symbol].items():
        print(f"  {info_key}: {info_value}")
    return lots_ticker

In [90]:
def take_dispersion_position(timestamp, remarks, ticker, lots):
    legs = []
    legs = Helpers.get_legs_ironfly_price(ticker, timestamp, lots, ticker.intention, moneyness_ironfly, price_factor)
    for leg in legs:
        key = f'{ticker.symbol}_{leg.id()}'
        if key not in ticker.tokens.keys():
            ticker.tokens[key] = TL.Token(ticker, leg.Instrument, leg.Strike, leg.LegName)
        token = ticker.tokens[key]
        token.add_position(timestamp, lots, leg.Position)
    ticker.take_position(timestamp, remarks, *legs)


In [91]:
class DispersionPTSL(TL.PTSLHandling):
    def __init__(self, profit_target, stop_loss, *tickers):
        super().__init__(profit_target, stop_loss, *tickers)

    def is_valid(self, timestamp):
        # if PTSL is active, update to make sure it's correct
        if self.active_ptsl:
            self.update_validity(timestamp)
        return not self.active_ptsl   
    
    def update_validity(self, timestamp):
        if timestamp.date() != self.triggered_at.date():
            self.reset()
            print(f"New day, PTSL for Intraday Strategy is resetted")
        # index = self.tickers[0]
        # for z in index.df_futures['zscore'].loc[self.triggered_at : timestamp]:
        #     if abs(z) <= epsilon:
        #         self.triggered_at = None
        #         self.active_ptsl = False  
        #         print(f"PTSL reverted back to mean (<= {epsilon}): Future Signals can now be used for trading")

In [92]:
def squareoff(time_ix, timestamp, remarks, logging = True):
    print("************  SQUARE OFF BEGINS  *********** ")
    # print("******  Information updated globally. TICKER WISE UPDATION BEGINS  ******")
    TL.squareoff_ticker(time_ix, timestamp, remarks, index, logging)
    for component in index.components.values():
        TL.squareoff_ticker(time_ix, timestamp, remarks, component, logging)
    print("* SQUARE OFF TRADES SAVED, TOKENS IN TICKERS UPDATED")
    print("************  SQUARE OFF COMPLETE  *********** ")


In [93]:
def isTodayAnyExpiry(timestamp, index):
    is_expiry = False
    is_expiry = is_expiry or index.is_expiry(timestamp)
    for component in index.components.values():
        is_expiry = is_expiry or component.is_expiry(timestamp)
    return is_expiry

In [94]:
def UpdateDispersionTickers(time_ix, timestamp, remarks, hedge, logging_token_update, logging_hedging):
    index_delta_threshold_per_lot = tell_delta_threshold_per_lot(index)
    TL.HandleUpdate(index, time_ix, timestamp, remarks, index_delta_threshold_per_lot, hedge, logging_token_update, logging_hedging)
    for component in index.components.values():
        component_delta_threshold_per_lot = tell_delta_threshold_per_lot(component)
        TL.HandleUpdate(component, time_ix, timestamp, remarks, component_delta_threshold_per_lot, hedge, logging_token_update, logging_hedging)

In [95]:
def zoom_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    portfolio = [index, *[component for component in index.components.values()]]
    performance = Helpers.zoom_tokens_performance_bar_by_bar(*portfolio, start=start, end=end)

    # for name, df in performance.items():
    #     if name != "Combined Summary":
    #         # df['Z-Score'] = index.df_futures['zscore'].loc[start:end]
    #         # df['IC'] = index.df_futures['ic'].loc[start:end]

    Plot.save_df_to_excel(performance, file_name)

In [96]:
def visualise_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    portfolio = [index, *[component for component in index.components.values()]]
    result_df = Helpers.get_summary_portfolio(*portfolio, start=start, end=end)

    for ticker in portfolio:
        result_df[f'{ticker.symbol} IV'] = ticker.get_iv_during(start, end)*100
        result_df[f'{ticker.symbol} Futures Price'] = ticker.get_futures_data(start, end)

    fig = Plot.plot_df(result_df, *(result_df.columns))
    Plot.save_plot(fig, file_name)

In [97]:
def save_dispersion_trades(start, end, file_name):
    trades_dict = {}
    portfolio = [index, *[comp for comp in index.components.values()]]
    trades_dict['All Trades'] = Helpers.get_trades_portfolio(*portfolio, start=start, end=end)
    for ticker in portfolio:
        trades_dict[ticker.symbol] = Helpers.get_trades_ticker(ticker, start=start, end=end)
    Plot.save_df_to_excel(trades_dict, file_name)

In [98]:
file = open(f'LOGS_Trading_Logic_for_{strategy_desc}.txt', 'w')
sys.stdout = file

index.reset_trades()
for component in index.components.values():
    component.reset_trades()

TrackPTSL = DispersionPTSL(profit_target, stop_loss, index, *index.components.values())

current_position = None

trade_start_date, trade_end_date, trade_count = None, None, 0

for time_ix, timestamp in enumerate(index.timestamps):
    print()
    print(f"Timestamp: {timestamp}")
    #######################################################################################################################################################
    if TL.check_existing_position(index):
        
        print(f"Existing Position Check: TRUE | Dispersion position: {current_position}")
        
        # 3:20 (Market Close) SQUAREOFF
        if exit_signal(timestamp):
            print("************  MARKET CLOSE  *********** ")
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            squareoff(time_ix, timestamp, f'3:20/Market Close Square Off', True)
            trade_end_date = timestamp
            trade_count+=1
            zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
            visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
            continue

        # PROFIT TARGET AND STOP LOSS SQUAREOFF
        if TrackPTSL.is_valid(timestamp) and TrackPTSL.status(timestamp) != PTSL.Valid:
            print("************  TrackPTSL Trigger hit *********** ")
            print(f"{TrackPTSL.nature} Square off at NetPnl of {TrackPTSL.pnl_last_trade}")
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            squareoff(time_ix, timestamp, f'{TrackPTSL.nature} Square Off', True)
            trade_end_date = timestamp
            trade_count+=1
            zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
            visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
            continue
            
        # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
        UpdateDispersionTickers(time_ix, timestamp, 'Delta Hedging using Synthetic Futures', True, True, True)
        print("========================================================================================================================================================================")
        continue
    #######################################################################################################################################################
    
    
    #######################################################################################################################################################
    if entry_signal(timestamp):
        print("************  ENTRY TIME REACHED  ************  ")
        UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
            continue

        if not TrackPTSL.is_valid(timestamp):
            print(f"{TrackPTSL.nature} was Triggered at {TrackPTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
            continue
        
        # try:
        lots_index = get_lots_for_entry(index, timestamp)
        if lots_index == 0:
            print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
            continue
        take_dispersion_position(timestamp, f'{ICView.name} IC', index, lots_index)
        
        for component in index.components.values():
            lots_component = get_lots_for_entry(component, timestamp)
            take_dispersion_position(timestamp, f'{ICView.name} IC', component, lots_component)
        
        TrackPTSL.fresh_trade(timestamp)
        current_position = ICView
        trade_start_date = timestamp
        print(f"{ICView} IC Trade executed")
        # except Exception as e:
        #     print(f"Couldn't Enter a {ICView} IC Trade Position: {e}")  

        continue
    #######################################################################################################################################################

    UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
    print("========================================================================================================================================================================")
    

file.close()
sys.stdout = original_stdout.value

C:\Users\vinayak\Desktop\Backtesting\Modules\Helpers.py:326: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\vinayak\Desktop\Backtesting\Modules\Helpers.py:320: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\vinayak\Desktop\Backtesting\Modules\Helpers.py:299: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfra

In [99]:
import importlib
importlib.reload(Plot)

<module 'Modules.Plot' from 'C:\\Users\\vinayak\\Desktop\\Backtesting\\Modules\\Plot.py'>

In [100]:
visualise_dispersion_trade(
    start=index.timestamps[0], 
    end=index.timestamps[-1], 
    file_name='PLOT_Complete_Period_Summary.html'
)

Timestamp: 2024-10-04 09:15:00
  IV Sanity Parameters || Spot Price(XX): 52305.1 | ATM Call Strike: 52300.0 | ATM Put Strike: 52300.0 | Symbol: BANKNIFTY
Error in Generating Greeks from Quantlib | root not bracketed: f[1e-07,4] -> [nan,nan]
  Greeks Sanity Parameters || spot(xx): 52305.1 | strike: 52300.0 | option price: nan | time to expiry: 26days | option type: Option.Put | expiry: 2024-10-30
  Could not Generate Greeks for BANKNIFTY
  Problem | root not bracketed: f[1e-07,4] -> [nan,nan]
Failed to Generate IV || Greeks from Call: SUCCESS, Greeks from Put: FAILURE
------------------------------------------
Timestamp: 2024-10-04 09:16:00
  IV Sanity Parameters || Spot Price(XX): 52198.95 | ATM Call Strike: 52200.0 | ATM Put Strike: 52200.0 | Symbol: BANKNIFTY
Successfully Generated IV | IV = 0.1445617122721662
  IV Sanity Values ||  | IV from Call: 14.783691009360098% | IV from Put: 14.12865144507314%
------------------------------------------
Timestamp: 2024-10-04 09:17:00
  IV Sani

In [101]:
save_dispersion_trades(
    start=index.df_futures.index[0], 
    end=index.df_futures.index[-1], 
    file_name=f'Info_Trades_for_{strategy_desc}.xlsx'
)

In [102]:
def metrics(fund_blocked):
    info_metrics = open(f'INFO_Metrics_for_{strategy_desc}.txt', 'w')
    sys.stdout = info_metrics

    print(f"TimeFrame Length: 6 months")
    print(f"Fundblocked in rs: {fund_blocked/100}")
    result_df.index = pd.to_datetime(result_df.index)
    result_df['date'] = result_df.index.date  # Create a new column for the date
    daily_profit = result_df.groupby('date')['running_pnl'].agg(lambda x: x.iloc[-1] - x.iloc[0])
    daily_profit.index = pd.to_datetime(result_df['date'].unique())
    
    ret = daily_profit.sum()/fund_blocked * 100
    print(f"Return: {round(ret)}%")
    
    sharpe_value = (daily_profit.mean() - risk_free_rate*fund_blocked/365)/daily_profit.std()
    print(f"Sharpe: {round(sharpe_value)}")
    
    peaks = result_df['running_pnl'].cummax()
    drawdowns = (result_df['running_pnl'] - peaks)/(peaks)
    max_drawdown = drawdowns.min()
    print(f"Drawdown: {round(max_drawdown)}%")
    monthly_data = daily_profit.resample('ME').sum()
    monthly_profit = monthly_data[monthly_data > 0].sum()
    monthly_loss = abs(monthly_data[monthly_data < 0].sum())
    if monthly_loss != 0:
        profit_loss_ratio = monthly_profit / monthly_loss
    else:
        profit_loss_ratio = float('inf')  # Avoid division by zero
    print("Monthly Profit/Loss Ratio:", round(profit_loss_ratio))
    
    windays = (result_df['running_pnl'] > 0).sum()
    lossdays = (result_df['running_pnl'] < 0).sum()
    if (windays + lossdays) > 0:
        win_ratio = windays / (windays + lossdays)
    else:
        win_ratio = 0
    print("Win Ratio:", round(win_ratio))
    
    streak = (result_df['running_pnl'] > 0).astype(int)  # 1 for winning days, 0 otherwise
    losing_streak = (result_df['running_pnl'] < 0).astype(int)  # 1 for losing days, 0 otherwise
    winning_streaks = streak.groupby((streak != streak.shift()).cumsum()).cumsum()
    losing_streaks = losing_streak.groupby((losing_streak != losing_streak.shift()).cumsum()).cumsum()
    print(f"Longest Winning Streak: {winning_streaks.max()}, Longest Losing Streak: {losing_streaks.max()}")

    sys.stdout = original_stdout.value
    info_metrics.close()

In [103]:
timestamp = pd.to_datetime('2024-11-04 09:27:00')
index.greeks(timestamp, Option.Call, 51800, True)

  Greeks Sanity Parameters
  >> Spot(xx): 51700.75
  >> Strike: 51800.0
  >> Option Price: 890.45
  >> Time to Expiry: 23days
  >> Option Type: Call
  >> Expiry: 2024-11-27
  Greeks Sanity Values (Per Option)
  >> IV: 0.18124804986590604
  >> Delta: 0.4922594628400123
  >> Gamma: 0.00016956672650771782
  >> Theta: -20.396617309729873
  >> Vega: 51.76576503536019
  >> Rho: 1547.5996432375812
